## Load Verification Dataset

In [ ]:
from clustering import LGKUtils

n_outputs = 5
verification_csv = '../data/latest/verification.csv'
X_orig, y_orig = LGKUtils.load_dataset(verification_csv, n_outputs)

## Generate Label Guided K-Means Regions

In [ ]:
from clustering import LGKClustering, LGKUtils

X, y = X_orig.copy(), y_orig.copy()
y = LGKUtils.reduce_classes(y)
X, y = remove_outliers(X, y, 10)
lgkmc = LGKClustering().fit(X, y, init_centroid='rand') # rand, first, none

# save regions
LGKUtils.save(lgkmc, outdir='../artifacts')

## Verify Regions

In [ ]:
import os
from clustering import LGKUtils
from robustness import verify_regions, save_verified_regions

# load the regions
lgkmc = LGKUtils.load('../artifacts/lgkm.pkl')
regions = lgkmc.get_regions(sort=True)
categories = regions.get_categories()

# grab a subset of regions to verify... (optional, you could try to verify them all)
subset_regions = regions[0:10]

# verify the regions
nnet_path = '../network/models/latest/model.nnet'
vregions = verify_regions(nnet_path, subset_regions, categories, eprec=0.0001, rpad=10, verbose=1, timeout=timeout)
save_verified_regions(vregions, outdir='../artifacts/')